In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

/home/speroulakis/.pyenv/versions/3.10.6/envs/Mental_Health_Risks/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-04 12:44:08.519645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733312648.537149   40685 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733312648.542282   40685 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 12:44:08.561034: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU

In [2]:
path = kagglehub.dataset_download("jessicali9530/kuc-hackathon-winter-2018")

In [3]:
df = pd.read_csv(os.path.join(path, 'drugsComTrain_raw.csv'))

In [4]:
def data_filter(df,uselfCount=0,min_length=30):

    df = df.dropna()
#     condition_distribution = df['condition'].value_counts()
#     best_conditions = condition_distribution.head(top_conditions)
#     top_conditions_list = best_conditions.index.tolist()
#     df = df[df['condition'].isin(top_conditions_list)]
    df = df[df['rating'].isin([1,2,3,8,9,10])]
    df['sentiment'] = 0
    df['sentiment'] = (df['rating'].isin([8,9,10])).astype(int)
    df = df[df['usefulCount']>uselfCount]
    df['review_length'] = df['review'].apply(lambda x: len(str(x).split()))
    df = df[df.review_length>=min_length]

    return df

In [5]:
df1 = data_filter(df)

In [6]:
df1.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sentiment,review_length
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,1,141
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,1,89
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,1,124
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,0,68
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5,0,30


In [7]:
y = df1['sentiment']

In [8]:
X = np.load('merged.npy')

In [9]:
X.shape

(109890, 128)

In [40]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(128,)),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [41]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,881 (105.00 KB)

 Trainable params: 26,881 (105.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
import logic.processing as lp

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [15]:
Xb, yb = lp.balance_dataset(X,y)

In [24]:
type(Xb)

pandas.core.series.Series

In [26]:
Xbb = np.stack(Xb.values)

In [30]:
from tensorflow.keras.callbacks import EarlyStopping

In [31]:
es = EarlyStopping(patience=20, restore_best_weights=True)

In [42]:
seb2 = model.fit(Xbb,yb, batch_size=32, epochs=500, validation_split=0.2)

Epoch 1/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6347 - loss: 0.6358 - val_accuracy: 0.6850 - val_loss: 0.5916
Epoch 2/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6760 - loss: 0.5986 - val_accuracy: 0.6963 - val_loss: 0.5788
Epoch 3/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6955 - loss: 0.5774 - val_accuracy: 0.7005 - val_loss: 0.5710
Epoch 4/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7027 - loss: 0.5715 - val_accuracy: 0.6804 - val_loss: 0.5929
Epoch 5/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7025 - loss: 0.5693 - val_accuracy: 0.7096 - val_loss: 0.5569
Epoch 6/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7123 - loss: 0.5592 - val_accuracy: 0.7133 - val_loss: 0.5555
Epoch 7/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7130 - loss: 0.5537 - val_accuracy: 0.7028 - val_loss: 0.5655
Epoch 8/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7243 - loss: 0

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7982 - loss: 0.4289 - val_accuracy: 0.7370 - val_loss: 0.5510
Epoch 51/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7977 - loss: 0.4293 - val_accuracy: 0.7369 - val_loss: 0.5420
Epoch 52/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7939 - loss: 0.4318 - val_accuracy: 0.7408 - val_loss: 0.5503
Epoch 53/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7992 - loss: 0.4268 - val_accuracy: 0.7377 - val_loss: 0.5436
Epoch 54/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7990 - loss: 0.4294 - val_accuracy: 0.7415 - val_loss: 0.5443
Epoch 55/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8031 - loss: 0.4180 - val_accuracy: 0.7393 - val_loss: 0.5471
Epoch 56/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8005 - loss: 0.4274 - val_accuracy: 0.7406 - val_loss: 0.5430
Epoch 57/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8017 - loss: 0.4252

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8270 - loss: 0.3796 - val_accuracy: 0.7524 - val_loss: 0.5668
Epoch 100/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8350 - loss: 0.3746 - val_accuracy: 0.7466 - val_loss: 0.5722
Epoch 101/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8292 - loss: 0.3754 - val_accuracy: 0.7450 - val_loss: 0.5793
Epoch 102/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8293 - loss: 0.3766 - val_accuracy: 0.7469 - val_loss: 0.5855
Epoch 103/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8314 - loss: 0.3737 - val_accuracy: 0.7471 - val_loss: 0.5810
Epoch 104/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8294 - loss: 0.3737 - val_accuracy: 0.7513 - val_loss: 0.5732
Epoch 105/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8340 - loss: 0.3713 - val_accuracy: 0.7507 - val_loss: 0.5801
Epoch 106/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8334 - loss:

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8463 - loss: 0.3470 - val_accuracy: 0.7543 - val_loss: 0.5966
Epoch 149/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8440 - loss: 0.3514 - val_accuracy: 0.7514 - val_loss: 0.5876
Epoch 150/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8452 - loss: 0.3466 - val_accuracy: 0.7593 - val_loss: 0.6038
Epoch 151/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8445 - loss: 0.3520 - val_accuracy: 0.7537 - val_loss: 0.5910
Epoch 152/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8457 - loss: 0.3470 - val_accuracy: 0.7538 - val_loss: 0.6049
Epoch 153/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8438 - loss: 0.3473 - val_accuracy: 0.7451 - val_loss: 0.5987
Epoch 154/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8440 - loss: 0.3457 - val_accuracy: 0.7490 - val_loss: 0.6049
Epoch 155/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8430 - loss:

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8582 - loss: 0.3230 - val_accuracy: 0.7567 - val_loss: 0.6093
Epoch 198/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8517 - loss: 0.3291 - val_accuracy: 0.7529 - val_loss: 0.6208
Epoch 199/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8600 - loss: 0.3212 - val_accuracy: 0.7515 - val_loss: 0.6303
Epoch 200/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8537 - loss: 0.3283 - val_accuracy: 0.7535 - val_loss: 0.6262
Epoch 201/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8551 - loss: 0.3289 - val_accuracy: 0.7580 - val_loss: 0.6145
Epoch 202/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8561 - loss: 0.3274 - val_accuracy: 0.7548 - val_loss: 0.6367
Epoch 203/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8569 - loss: 0.3268 - val_accuracy: 0.7487 - val_loss: 0.6521
Epoch 204/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8529 - loss:

...

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8695 - loss: 0.3033 - val_accuracy: 0.7614 - val_loss: 0.6536
Epoch 296/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8693 - loss: 0.3031 - val_accuracy: 0.7619 - val_loss: 0.6403
Epoch 297/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8727 - loss: 0.3004 - val_accuracy: 0.7605 - val_loss: 0.6558
Epoch 298/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8685 - loss: 0.3068 - val_accuracy: 0.7648 - val_loss: 0.6539
Epoch 299/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8704 - loss: 0.2984 - val_accuracy: 0.7599 - val_loss: 0.6400
Epoch 300/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8680 - loss: 0.3033 - val_accuracy: 0.7576 - val_loss: 0.6604
Epoch 301/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8687 - loss: 0.3021 - val_accuracy: 0.7578 - val_loss: 0.6398
Epoch 302/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8712 - loss:

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8734 - loss: 0.2990 - val_accuracy: 0.7638 - val_loss: 0.6511
Epoch 345/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8755 - loss: 0.2913 - val_accuracy: 0.7651 - val_loss: 0.6540
Epoch 346/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8723 - loss: 0.2960 - val_accuracy: 0.7622 - val_loss: 0.6505
Epoch 347/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8757 - loss: 0.2961 - val_accuracy: 0.7582 - val_loss: 0.6531
Epoch 348/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8734 - loss: 0.2953 - val_accuracy: 0.7569 - val_loss: 0.6579
Epoch 349/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8775 - loss: 0.2893 - val_accuracy: 0.7626 - val_loss: 0.6417
Epoch 350/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8739 - loss: 0.2964 - val_accuracy: 0.7625 - val_loss: 0.6460
Epoch 351/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8723 - loss:

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8769 - loss: 0.2921 - val_accuracy: 0.7639 - val_loss: 0.6608
Epoch 394/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8771 - loss: 0.2896 - val_accuracy: 0.7576 - val_loss: 0.6619
Epoch 395/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8759 - loss: 0.2935 - val_accuracy: 0.7605 - val_loss: 0.6729
Epoch 396/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8777 - loss: 0.2851 - val_accuracy: 0.7638 - val_loss: 0.6721
Epoch 397/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8754 - loss: 0.2921 - val_accuracy: 0.7602 - val_loss: 0.6623
Epoch 398/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8761 - loss: 0.2917 - val_accuracy: 0.7635 - val_loss: 0.6457
Epoch 399/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8781 - loss: 0.2895 - val_accuracy: 0.7651 - val_loss: 0.6534
Epoch 400/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8795 - loss:

1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8781 - loss: 0.2876 - val_accuracy: 0.7618 - val_loss: 0.6825
Epoch 443/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8792 - loss: 0.2876 - val_accuracy: 0.7677 - val_loss: 0.6668
Epoch 444/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8789 - loss: 0.2828 - val_accuracy: 0.7633 - val_loss: 0.6674
Epoch 445/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8799 - loss: 0.2839 - val_accuracy: 0.7659 - val_loss: 0.6561
Epoch 446/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8777 - loss: 0.2866 - val_accuracy: 0.7650 - val_loss: 0.6628
Epoch 447/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8800 - loss: 0.2818 - val_accuracy: 0.7672 - val_loss: 0.6634
Epoch 448/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8797 - loss: 0.2839 - val_accuracy: 0.7638 - val_loss: 0.6759
Epoch 449/500
1400/1400 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8805 - loss:

KeyboardInterrupt: 